In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!pip install ultralytics lxml opencv-python easyocr scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 28.7 MB/s eta 0:00:00


In [5]:
!pip install -U ultralytics
!pip install easyocr


In [ ]:
pip install --upgrade kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: kagglehub
    Found existing installation: kagglehub 0.3.13
    Uninstalling kagglehub-0.3.13:
      Successfully uninstalled kagglehub-0.3.13


In [ ]:
import kagglehub
import shutil
import os

# Define destination inside Google Drive
dest_path = "/content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset"

# Download dataset to default KaggleHub cache location first
# This bypasses the issue with 'path' parameter directly to the API
print("Downloading dataset to temporary location...")
temp_download_path = kagglehub.dataset_download(
    "saisirishan/indian-vehicle-dataset"
)

print(f"Dataset downloaded to temporary location: {temp_download_path}")

# Ensure the final destination directory exists
os.makedirs(dest_path, exist_ok=True)

# Move the contents from the temporary download path to the final destination
print(f"Moving dataset contents from {temp_download_path} to {dest_path}...")

# Copy contents (files and subdirectories) from temp_download_path to dest_path
for item in os.listdir(temp_download_path):
    s = os.path.join(temp_download_path, item)
    d = os.path.join(dest_path, item)
    if os.path.isdir(s):
        shutil.copytree(s, d, dirs_exist_ok=True) # dirs_exist_ok=True for Python 3.8+
    else:
        shutil.copy2(s, d) # copy2 preserves metadata

# Update the 'path' variable to reflect the final location for the print statement
path = dest_path

print("Dataset download and move complete!")
print("Dataset available at:", path)

Using Colab cache for faster access to the 'indian-vehicle-dataset' dataset.
Dataset downloaded to temporary location: /kaggle/input/indian-vehicle-dataset
Moving dataset contents from /kaggle/input/indian-vehicle-dataset to /content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset...
Dataset download and move complete!
Dataset available at: /content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset


In [ ]:
import os, shutil

BASE = "/content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset"

IMG_OUT = f"{BASE}/all_images"
XML_OUT = f"{BASE}/all_xml"

os.makedirs(IMG_OUT, exist_ok=True)
os.makedirs(XML_OUT, exist_ok=True)

SKIP_FOLDERS = ["all_images", "all_xml", "images", "labels"]

for root, _, files in os.walk(BASE):
    # Skip output folders
    if any(skip in root for skip in SKIP_FOLDERS):
        continue

    for file in files:
        src = os.path.join(root, file)

        if file.lower().endswith(".jpg"):
            dst = os.path.join(IMG_OUT, file)
        elif file.lower().endswith(".xml"):
            dst = os.path.join(XML_OUT, file)
        else:
            continue

        # Copy only if destination file doesn't already exist
        if not os.path.exists(dst):
            shutil.copy(src, dst)

print("✅ Images and XML files collected safely")


✅ Images and XML files collected safely


In [ ]:
import os

BASE = "/content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset"
IMG_OUT = f"{BASE}/all_images"
XML_OUT = f"{BASE}/all_xml"

images = set([f.replace(".jpg","") for f in os.listdir(IMG_OUT) if f.endswith(".jpg")])
xmls   = set([f.replace(".xml","") for f in os.listdir(XML_OUT) if f.endswith(".xml")])

missing_xml = images - xmls
missing_img = xmls - images

print("Images without XML:", len(missing_xml))
print("XML without Image:", len(missing_img))


Images without XML: 0
XML without Image: 359


In [ ]:
for name in missing_xml:
    os.remove(f"{IMG_OUT}/{name}.jpg")

for name in missing_img:
    os.remove(f"{XML_OUT}/{name}.xml")

print("✅ Unmatched files removed")


✅ Unmatched files removed


In [ ]:
import xml.etree.ElementTree as ET

bad_files = []

for xml_file in os.listdir(XML_OUT):
    try:
        tree = ET.parse(os.path.join(XML_OUT, xml_file))
        root = tree.getroot()

        w = int(root.find("size/width").text)
        h = int(root.find("size/height").text)

        for obj in root.findall("object"):
            box = obj.find("bndbox")
            xmin = int(box.find("xmin").text)
            ymin = int(box.find("ymin").text)
            xmax = int(box.find("xmax").text)
            ymax = int(box.find("ymax").text)

            if xmin < 0 or ymin < 0 or xmax > w or ymax > h:
                bad_files.append(xml_file)

    except:
        bad_files.append(xml_file)

print("Bad XML files:", len(set(bad_files)))


Bad XML files: 0


In [ ]:
for xml in set(bad_files):
    name = xml.replace(".xml","")
    if os.path.exists(f"{XML_OUT}/{xml}"):
        os.remove(f"{XML_OUT}/{xml}")
    if os.path.exists(f"{IMG_OUT}/{name}.jpg"):
        os.remove(f"{IMG_OUT}/{name}.jpg")

print("✅ Corrupted annotations removed")


✅ Corrupted annotations removed


In [ ]:
import xml.etree.ElementTree as ET

LABELS = f"{BASE}/labels"
os.makedirs(LABELS, exist_ok=True)

classes = ["number_plate"]

for xml_file in os.listdir(XML_OUT):
    tree = ET.parse(os.path.join(XML_OUT, xml_file))
    root = tree.getroot()

    w = int(root.find("size/width").text)
    h = int(root.find("size/height").text)

    label_path = os.path.join(LABELS, xml_file.replace(".xml",".txt"))

    with open(label_path, "w") as f:
        for obj in root.findall("object"):
            cls = obj.find("name").text.lower()
            if cls not in classes:
                continue

            box = obj.find("bndbox")
            xmin = int(box.find("xmin").text)
            xmax = int(box.find("xmax").text)
            ymin = int(box.find("ymin").text)
            ymax = int(box.find("ymax").text)

            x = ((xmin + xmax) / 2) / w
            y = ((ymin + ymax) / 2) / h
            bw = (xmax - xmin) / w
            bh = (ymax - ymin) / h

            f.write(f"0 {x} {y} {bw} {bh}\n")

print("✅ XML → YOLO conversion completed")


✅ XML → YOLO conversion completed


In [ ]:
from sklearn.model_selection import train_test_split
import shutil

images = [f for f in os.listdir(IMG_OUT) if f.endswith(".jpg")]

train, val = train_test_split(images, test_size=0.2, random_state=42)

for split, data in zip(["train","val"], [train,val]):
    os.makedirs(f"{BASE}/images/{split}", exist_ok=True)
    os.makedirs(f"{BASE}/labels/{split}", exist_ok=True)

    for img in data:
        shutil.copy(f"{IMG_OUT}/{img}", f"{BASE}/images/{split}/{img}")
        shutil.copy(f"{LABELS}/{img.replace('.jpg','.txt')}",
                    f"{BASE}/labels/{split}")

print("✅ Train/Validation split ready")


✅ Train/Validation split ready


In [ ]:
BASE = "/content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset"

yaml_path = f"{BASE}/data.yaml"

with open(yaml_path, "w") as f:
    f.write(
f"""path: {BASE}
train: images/train
val: images/val

nc: 1
names: ['number_plate']
"""
    )

print("✅ data.yaml created at:", yaml_path)


✅ data.yaml created at: /content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset/data.yaml


In [ ]:
!cat /content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset/data.yaml


path: /content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset
train: images/train
val: images/val

nc: 1
names: ['number_plate']


In [ ]:
!ls /content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset/images


train  val


In [ ]:
!ls /content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset/images/train | head
!ls /content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset/labels/train | head


00b42b2c-f193-4863-b92c-0245cbc816da___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_Nissan-Terrano-Petrol-Review-Images-Black-Front-Angle.jpg
07aaab79-71ee-4ea3-a9e6-640191183947___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_1208484d1392541449-nissan-terrano-official-review-img_20140215_181708.jpg
0850c175-0b8d-47f2-801c-e29f1dbdb367___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_571.jpg
0a0d1748-48cd-4114-90cb-b5baf0b3cbe4___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_147274518_15141875973_large.jpg
0a720df9-e4ef-4e44-8c13-b39b9be8444d___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_Nissan-Terrano-6.jpg
0b77e2a4-34db-486c-8d87-46186f5b7b88___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_nissan-terrano-diesel-review-motorzest-04.jpg
1a0da3dc-6667-4092-bc2a-a541a06f1e90___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_Nissan-Terrano9.jpg
1e7151e6-8626-4fb2-b7e8-0fb6a8fde713___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_a19be5870f8f2593c994c465a691477e_555X416_1.jpg
2430703d-0fb3-4eb2-9765-4f9301f232cd___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_Terrano-Du

In [ ]:
import shutil, os

LABELS = f"{BASE}/labels"
if os.path.exists(LABELS):
    shutil.rmtree(LABELS)
os.makedirs(LABELS, exist_ok=True)

print("✅ Old empty labels removed")


✅ Old empty labels removed


In [ ]:
import xml.etree.ElementTree as ET
import os

BASE = "/content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset"
XML_OUT = f"{BASE}/all_xml"
LABELS = f"{BASE}/labels"

os.makedirs(LABELS, exist_ok=True)

for xml_file in os.listdir(XML_OUT):
    tree = ET.parse(os.path.join(XML_OUT, xml_file))
    root = tree.getroot()

    w = int(root.find("size/width").text)
    h = int(root.find("size/height").text)

    label_path = os.path.join(LABELS, xml_file.replace(".xml", ".txt"))

    with open(label_path, "w") as f:
        for obj in root.findall("object"):
            # ✅ IGNORE obj.find("name").text (plate number)
            box = obj.find("bndbox")

            xmin = int(box.find("xmin").text)
            ymin = int(box.find("ymin").text)
            xmax = int(box.find("xmax").text)
            ymax = int(box.find("ymax").text)

            x_center = ((xmin + xmax) / 2) / w
            y_center = ((ymin + ymax) / 2) / h
            bw = (xmax - xmin) / w
            bh = (ymax - ymin) / h

            # ✅ Always class 0 = number_plate
            f.write(f"0 {x_center} {y_center} {bw} {bh}\n")

print("✅ XML → YOLO conversion FIXED")


✅ XML → YOLO conversion FIXED


In [ ]:
from sklearn.model_selection import train_test_split
import shutil, os

IMG_OUT = f"{BASE}/all_images"
LABELS = f"{BASE}/labels"

images = [f for f in os.listdir(IMG_OUT) if f.endswith(".jpg")]

train, val = train_test_split(images, test_size=0.2, random_state=42)

for split, data in zip(["train","val"], [train,val]):
    os.makedirs(f"{BASE}/images/{split}", exist_ok=True)
    os.makedirs(f"{BASE}/labels/{split}", exist_ok=True)

    for img in data:
        shutil.copy(f"{IMG_OUT}/{img}", f"{BASE}/images/{split}/{img}")
        shutil.copy(f"{LABELS}/{img.replace('.jpg','.txt')}",
                    f"{BASE}/labels/{split}")

print("✅ Train/Validation split recreated")


✅ Train/Validation split recreated


In [11]:
from ultralytics import YOLO

model = YOLO("yolov8m.pt")

model.train(
    data="/content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset/data.yaml",
    epochs=20,
    imgsz=640,
    batch=16,
    lr0=0.003,        # lower learning rate
    lrf=0.01,         # final LR fraction
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,
    box=7.5,          # ↑ box loss importance
    cls=0.5,          # ↓ class loss (single class)
    dfl=1.5,
    patience=15
)


Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.003, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a94f1710bc0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [13]:
import shutil
import os

SRC_DIR = "/content/runs/detect/train"
DEST_DIR = "/content/drive/MyDrive/ANPR_Model_Full"

os.makedirs(DEST_DIR, exist_ok=True)

shutil.copytree(SRC_DIR, DEST_DIR, dirs_exist_ok=True)

print("✅ Full YOLO training folder saved to Google Drive")


✅ Full YOLO training folder saved to Google Drive


In [15]:
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/ANPR_Model_Full/weights/best.pt")
model.info()


Model summary: 170 layers, 25,856,899 parameters, 0 gradients, 79.1 GFLOPs


(170, 25856899, 0, 79.0656)